In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, WeekdayLocator,\
    DayLocator, MONDAY,date2num,num2date,AutoDateLocator
from matplotlib.finance import quotes_historical_yahoo_ohlc, candlestick_ohlc,candlestick2_ochl,volume_overlay3

from stockapp import models as stkmd
from dataapp import models as dtamd
from dataapp import tasks as dtatks
from dataapp import libs as dtalibs
from featureapp import libs as ftlibs
from featureapp import models as ftmd
from stockapp import tasks as stktks
from stockapp import libs as stklibs
import featureapp.models as ftmd
import featureapp.tasks as fttks
import queryapp.models as qrymd
import queryapp.tasks as qrytks

import charts.chartservers.libs as chservlibs
import charts.libs as chlibs

from datascience import models as dtscmd
from datascience import tasks as dtsctks
from datascience import libs as dtsclibs


import featureapp as ftapp
import utility as uty
from utility import models as utymd
import itertools as itt
import multiprocessing as mp
from django.db import connection,connections
from django.db import reset_queries
import time 
import pandas as pd
import numpy as np
import inspect
import imp
import datetime
from talib.abstract import *
import utility.models as utmd
import stockapp.libs as stklib
from utility import codemanager as cdmng
from utility import maintenance as mnt
import os 
import json
from django.contrib.auth.models import AnonymousUser
import threading

stk=stkmd.Stockmeta.objects.get(Symbol='TSLA')
Fromdate=pd.datetime(2008,1,1)
Todate=pd.datetime.today()
Trange=pd.date_range(Fromdate,Todate)
Trange=[T.date() for T in Trange if T.weekday()<=4]

import json
# fttks.computefeatuers(stk.id,Trange)

/usr/local/lib/python2.7/dist-packages/matplotlib/cbook.py:136: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)
/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This 

# Charting

In [ ]:
entries=[
    {'Symbol':'TSLA','TF':pd.datetime(2012,1,1).date(),'T0':pd.datetime(2011,1,1).date()  },
    {'Symbol':'AAPL','TF':pd.datetime(2012,1,1).date(),'T0':pd.datetime(2011,1,1).date()  }
]
chservlibs.request_db_charts(entries,5003)
# img=chlibs.CurrentByFutureChart_bydb(entries[0]['T0'],entries[0]['TF'],entries[0]['Symbol'],indicatorlist=(),pricecols=(),querycols=(),featcols=())


# Running Features

In [ ]:
print "---------------- features-------------------"
featurecodes=ftmd.FeatureComputeCode.objects.all()
computecode=featurecodes[0]
computeclass=computecode.importcomputeclass()
CF=computeclass(stk.id,Trange)
CF.computeall(skipdone=True)
# CF.saveall()
print CF.getfeaturelist()

CF.df=CF.addindicators(CF.df,[
        {'name':'SMAstd','timeperiod':20,'colname':'SMAstd20'},
        {'name':'EMAstd','timeperiod':8,'colname':'EMAstd8'},
    ])

# Running Queries

In [ ]:
querycodes=qrymd.QueryComputeCode.objects.all()
computecode=querycodes[0]
computeclass=computecode.importcomputeclass()
CQ=computeclass(stk.id,Trange)
CQ.computeall(skipdone=True)
CQ.saveall()

In [ ]:
CQ.OutcomeCharts()

In [ ]:
CQ.getquerylist()

In [ ]:
CQ.chartfeatures(addpricecols=(),ip=5562,
addfeatcols=[
    ['CCI5','CCI50'],
    ['PastPROFIT10days','PastLOSS10days'],['FutPROFIT10days','FutLOSS10days']
],
addquerycols=[
    'CCICHERRIES',
]
)

# Registering function

In [ ]:
from datascience import libs as dtsclibs
import pandas as pd

@dtsclibs.register_compfunc(RequiredImports=['import pandas as pd','from dataapp import libs as dtalibs'],overwrite_if_exists=False)
def extractdataset(data_id,Symbol):
    """
    @funcName : test
    @input x : an int
    @output df : pd.DataFrame, some random 2 by 2 
    @description : takes an int and then returns a dummy dataframe. This is just for testing purposes
    @Source : 
    def extractdataset(data_id,Symbol):
        window=60
        window_fut=30
        Tfs=map(lambda x: ( (x.date()-pd.Dateoffset(window)).date(),x.date(), (x.date()+pd.Dateoffset(window_fut)).date() ),
                pd.date_range(start=pd.datetime(2010,1,1),end=pd.datetime.today(),freq='W-MON') )

        N=len(Tfs)
        dfinstants=pd.DataFrame({'T0':map(lambda x: x[0],Tfs),'TF':map(lambda x: x[1],Tfs),'Symbol':[Symbol]*N})
        X,X_Meta=dtalibs.Getbatchdata(dfinstants)

        dfinstants=pd.DataFrame({'T0':map(lambda x: x[1],Tfs),'TF':map(lambda x: x[2],Tfs),'Symbol':[Symbol]*N})
        Y,Y_Meta=dtalibs.Getbatchdata(dfinstants)

        shard=dtscmd.DataShard(Data__id=data_id)
        shard.Info['X_Meta']=X_Meta
        shard.Info['Y_Meta']=Y_Meta
        shard.save()

        np.savez_compressed(shard.shardpath(),X=X,Y=Y)
    """
    
    window=60
    window_fut=30
    Tfs=map(lambda x: ( (x.date()-pd.Dateoffset(window)).date(),x.date(), (x.date()+pd.Dateoffset(window_fut)).date() ),
            pd.date_range(start=pd.datetime(2010,1,1),end=pd.datetime.today(),freq='W-MON') )
    
    N=len(Tfs)
    dfinstants=pd.DataFrame({'T0':map(lambda x: x[0],Tfs),'TF':map(lambda x: x[1],Tfs),'Symbol':[Symbol]*N})
    X,X_Meta=dtalibs.Getbatchdata(dfinstants)
    
    dfinstants=pd.DataFrame({'T0':map(lambda x: x[1],Tfs),'TF':map(lambda x: x[2],Tfs),'Symbol':[Symbol]*N})
    Y,Y_Meta=dtalibs.Getbatchdata(dfinstants)
    
    shard=dtscmd.DataShard(Data__id=data_id)
    shard.Info['X_Meta']=X_Meta
    shard.Info['Y_Meta']=Y_Meta
    shard.save()
    
    np.savez_compressed(shard.shardpath(),X=X,Y=Y)

extractdataset.id


# Creating initial Stock price dataset

In [1]:
from datascience import models as dtscmd
from datascience import tasks as dtsctks
from datascience import libs as dtsclibs

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
projid,dataid=dtsclibs.register_dataset(project_Name="PredictReturn",project_Info={'description': "Data taken on every Monday. 360 days back and 60 days forward"},
                                        Datatype='RawProcessed',GroupName="AllStocks",tag="1",
                                        data_format='npz',Modeltype='Regression',
                                        TransformedFromDataId=None,TransFuncId=None, use_project_ifexists=True)

dtsctks.CreateStockData_2(360,60,dataid,None)

In [ ]:
print dtscmd.Data.objects.filter(Project__id=7,GroupName="AllStocks",tag="1",Datatype='RawProcessed',
                           Dataformat='npz',Modeltype='Regression')
data = dtscmd.Data.objects.get(Project__id=7,GroupName="AllStocks",tag="1",Datatype='RawProcessed',
                           Dataformat='npz',Modeltype='Regression')

data.datapath()

In [ ]:
dtscmd.DataShard.objects.filter(Data=data).delete()

In [ ]:
dtscmd.DataShard.objects.all()

# Functions on shards

In [25]:
from __future__ import division
from datascience import models as dtscmd
from datascience import tasks as dtsctks
from datascience import libs as dtsclibs
from celery import group
import time
import pickle as pkl


@dtsclibs.register_compfunc(Group='Cleanup',overwrite_if_exists=True)
def CleanData(shardId):
    """
    @Description: remove shards with many Nan samples. Remove samples with many Nans
    @Source
    import pandas as pd
    import numpy as np
    import datascience.models as dtscmd
    
    
    
    shard=dtscmd.DataShard.objects.get(id=shardId)
    X,Y,Meta=shard.getdata()
    
    # remove samples with lot of nans
    Nsamples=X.shape[0]
    Tsteps=X.shape[1]
    Nfeat=X.shape[2]
    volumecols=[4,11,12]
    colsX=list( Meta['MetaX']['columns'] )
    colsY=list( Meta['MetaY']['columns'] )
    delsample=[]
    for i in range(Nsamples):
        dfX=pd.DataFrame(X[i,:,:],columns=colsX)
        dfY=pd.DataFrame(X[i,:,:],columns=colsY)
        if np.sum(pd.isnull(dfX['Close'].values).astype(float))/len(dfX)>=0.5:
            delsample.append(i)
    
    X=np.delete(X,delsample,axis=0)
    Y=np.delete(Y,delsample,axis=0)
    
    # if not more samples then delete the shard
    if X.shape[0]/(Nsamples*1.0) <0.4:
        print "----------------------"
        print str(shardId)+"   "+ str(X.shape[0]/(Nsamples*1.0))
        print "----------------------"
        print "delete shaord id = "+str(shard.id)
        shard.delete()
    elif X.shape[0]!=Nsamples:
        shard.savedata(X=X,Y=Y,Meta=Meta)
    else:
        pass
    """
    
    import pandas as pd
    import numpy as np
    import datascience.models as dtscmd
    
    
    
    shard=dtscmd.DataShard.objects.get(id=shardId)
    X,Y,Meta=shard.getdata()
    
    # remove samples with lot of nans
    Nsamples=X.shape[0]
    Tsteps=X.shape[1]
    Nfeat=X.shape[2]
    volumecols=[4,11,12]
    colsX=list( Meta['MetaX']['columns'] )
    colsY=list( Meta['MetaY']['columns'] )
    delsample=[]
    for i in range(Nsamples):
        dfX=pd.DataFrame(X[i,:,:],columns=colsX)
        dfY=pd.DataFrame(X[i,:,:],columns=colsY)
        if np.sum(pd.isnull(dfX['Close'].values).astype(float))/len(dfX)>=0.5:
            delsample.append(i)
    
    X=np.delete(X,delsample,axis=0)
    Y=np.delete(Y,delsample,axis=0)
    
    # if not more samples then delete the shard
    if X.shape[0]/(Nsamples*1.0) <0.4:
        print "----------------------"
        print str(shardId)+"   "+ str(X.shape[0]/(Nsamples*1.0))
        print "----------------------"
        print "delete shaord id = "+str(shard.id)
        shard.delete()
    elif X.shape[0]!=Nsamples:
        shard.savedata(X=X,Y=Y,Meta=Meta)
    else:
        pass
    
# dtsctks.applyfunc2data(CleanData.id,6)


over writing previous function
saving function :  CleanData
function id =  20


In [26]:
dtscmd.DataShard.objects.filter(Data__id=6).count()

6548

# Creating derived DataSet

In [ ]:
@dtsclibs.register_compfunc(Group='Transformer',overwrite_if_exists=True)
def StandardizeData_1(X,Y,Meta):

    # next normalize the volume to 0-1
    Nsamples=X.shape[0]
    Tsteps=X.shape[1]
    Nfeat=X.shape[2]
    volumecols=[4,11,12]
    colsX=list( Meta['MetaX']['columns'] )
    for i in range(Nsamples):
        dfX=pd.DataFrame(X[i,:,:],columns=colsX)


register_dataset(tag='DerivedFromId6',TransformedFromDataId=6,TransFuncId=StandardizeData_1.id )


In [3]:
from datascience import models as dtscmd
dtscmd.Project.objects.all()
# data=dtscmd.Data.objects.filter(Project=pr)[0]

<QuerySet [<Project: PredictReturn>]>

In [ ]:
# data.delete()

In [8]:
shard=dtscmd.DataShard.objects.get(id=14342)
X,Y,Meta=shard.getdata()

In [9]:
req=list( Meta['MetaY']['columns'] )
print req

['Close', 'Open', 'High', 'Low', 'Volume', 'Symbol', 'SMA10', 'SMA20', 'SMA50', 'SMA100', 'SMA200', 'VolSMA10', 'VolSMA20', 'EMA8', 'EMA20']


In [73]:
d={cc : 1 for cc in Meta['MetaX']['columns']}
curr=sorted( d.keys() )
print curr

['Close', 'EMA20', 'EMA8', 'High', 'Low', 'Open', 'SMA10', 'SMA100', 'SMA20', 'SMA200', 'SMA50', 'Symbol', 'VolSMA10', 'VolSMA20', 'Volume']


In [17]:
import numpy as np
sum(np.isnan(X[0,:,1].astype(float)).astype(int))/360

0.31111111111111112

In [55]:
np.delete(X,[1,2],axis=0).shape

(392, 360, 15)

In [110]:
p=np.zeros(15)
for i in range(15):
    p[i]=curr.index(req[i])
p=p.astype(int)
p

array([ 0,  5,  3,  4, 14, 11,  6,  8, 10,  7,  9, 12, 13,  2,  1])

In [19]:
# df=pd.DataFrame(X[2,:,:],columns=req)
import pandas as pd
dfX=pd.DataFrame(X[2,:,:],columns=req)
print np.sum(pd.isnull(dfX['Close'].values).astype(float))/len(dfX)>=0.5


False


In [40]:
df

Close  Open  High   Low      Volume Symbol  SMA10   SMA20   SMA50  SMA100  \
0     NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
1     NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
2     NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
3     NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
4     NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
5     NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
6     NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
7     NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
8     NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
9     NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
10    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
11    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
12    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
13    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
14    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
15    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
16    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
17    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
18    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
19    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
20    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
21    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
22    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
23    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
24    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
25    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
26    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
27    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
28    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
29    NaN   NaN   NaN   NaN         NaN    NaN    NaN     NaN     NaN     NaN   
..    ...   ...   ...   ...         ...    ...    ...     ...     ...     ...   
330  2.33   2.4  2.48  2.31   0.0122408   ROYL  2.473  2.5325   2.455  2.2927   
331  2.41   2.4  2.48  2.35   0.0106096   ROYL  2.453  2.5215  2.4612  2.2938   
332  2.39  2.43  2.44  2.39  0.00495021   ROYL  2.429   2.514  2.4674  2.2949   
333  2.44  2.41  2.45  2.36  0.00651045   ROYL  2.416  2.5105   2.474  2.2968   
334  2.46  2.44  2.46  2.37  0.00365947   ROYL  2.416   2.496  2.4812  2.2985   
335   2.6  2.45  2.69  2.41    0.026751   ROYL  2.422   2.497    2.49  2.3015   
336  2.59   2.6  2.64  2.51  0.00727639   ROYL  2.436  2.5035  2.5012  2.3043   
337  2.64  2.62  2.75  2.61   0.0200987   ROYL  2.461   2.512  2.5136  2.3084   
338  2.71  2.64  2.74  2.62    0.012794   ROYL  2.493  2.5195   2.528  2.3128   
339  2.68  2.69  2.73  2.65  0.00663811   ROYL  2.525   2.521  2.5406   2.319   
340  2.75  2.66  3.04  2.65    0.066452   ROYL  2.567    2.52   2.554  2.3251   
341  2.69  2.77  2.89  2.69   0.0220561   ROYL  2.595   2.524   2.566  2.3315   
342  2.72  2.75   2.8   2.7  0.00811325   ROYL  2.628  2.5285   2.579  2.3378   
343  2.88   2.8  2.94  2.74   0.0335168   ROYL  2.672   2.544  2.5956  2.3454   
344  2.81  2.92  2.92   2.7   0.0198576   ROYL  2.707  2.5615  2.6116  2.3507   
345  2.78   2.8  2.87  2.75  0.00822672   ROYL  2.725  2.5735  2.6238  2.3564   
346  2.72  2.79  2.86  2.71   0.0116876   ROYL  2.738   2.587   2.614  2.3621   
347  2.66   2.7  2.73  2.66  0.0080

In [33]:
np.nanmean(X[2,:,4].astype(float))

152170.56451612903

In [104]:
H[:,:][-1]

array([83.76, 83.90806954350671, 84.03642943411059, 84.48, 83.62, 84.11,
       84.24000000000011, 79.3874999999999, 84.70999999999984,
       72.04099999999991, 81.70200000000004, u'ESS', 381550.0, 409760.0,
       181800.0], dtype=object)

In [101]:
H[:,p][-1]

array([83.76, 181800.0, 409760.0, 84.03642943411059, 84.48,
       83.90806954350671, 84.24000000000011, 72.04099999999991,
       79.3874999999999, 81.70200000000004, 84.70999999999984, 84.11,
       u'ESS', 381550.0, 83.62], dtype=object)

In [58]:
X[1,-1,:]

array([83.76, 83.90806954350671, 84.03642943411059, 84.48, 83.62, 84.11,
       84.24000000000011, 79.3874999999999, 84.70999999999984,
       72.04099999999991, 81.70200000000004, u'ESS', 381550.0, 409760.0,
       181800.0], dtype=object)

In [32]:
np.isnan(X[1,:,:])

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [5]:
Y.shape

(394, 60, 15)

In [28]:
Z=np.stack((X.transpose((0,2,1)),Y.transpose((0,2,1))),axis=1)

ValueError: all input arrays must have the same shape

In [136]:
Y[1,0,:]

array([83.76, 83.90806954350671, 84.03642943411059, 84.48, 83.62, 84.11,
       84.24000000000011, 79.3874999999999, 84.70999999999984,
       72.04099999999991, 81.70200000000004, u'ESS', 381550.0, 409760.0,
       181800.0], dtype=object)